# Best Model
The purpose of this notebook is to take the top performing models run them on what we believe to be the most effective data. We then search through the hyperparameter space of the models to try and optimise F1 measure

In [ ]:
import pandas as pd
import numpy as np
from os import path
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    average_precision_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    accuracy_score,
)
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import shap
from hyperopt import fmin, tpe, hp
from tabpfn import TabPFNClassifier
from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNClassifier

## Load and Process Data

In [73]:
data_file = path.join("..", "data", "zoonosis_dataset_full.csv")
target_column = "label"
columns_to_keep = [
    "label",
    "DPC_FS_HA",
    "CTDC_charge.G1_PB2",
    "ATTCC_HA",
    "TG_NS1",
    "CTDC_hydrophobicity_ARGP820101.G2_PB2",
    "CAC_HA",
    "DPC_MV_M1",
    "CTDT_hydrophobicity_ZIMJ680101.Tr1331_PB1",
    "CTDC_normwaalsvolume.G1_PB2",
    "TA_NP",
    "CTDT_secondarystruct.Tr1221_NS1",
    "PAAC_Xc1.Y_NA",
    "AAGGC_M1",
    "CTDC_hydrophobicity_CASG920101.G1_PB2",
    "ATTG_PA",
    "TGATAC_NS1",
    "PAAC_Xc1.R_PB2",
    "CTDD_secondarystruct.1.residue100_HA",
    "TT_PA",
    "CTDC_polarizability.G1_PA",
    "CTDT_solventaccess.Tr2332_PA",
    "CTDC_hydrophobicity_ZIMJ680101.G1_PB1",
    "CTCG_PB1",
    "CTDT_normwaalsvolume.Tr1331_NA",
    "AC_NA",
    "GTC_NP",
]

In [74]:
def prepare_dataframe_for_ml(df, target_column=None, one_hot_encode=True):
    """
    Prepare a pandas DataFrame for machine learning algorithms.
    - Normalizes numerical features
    - Optionally one-hot encodes categorical features
    - Optionally separates target variable

    Parameters:
    -----------
    df : pandas.DataFrame
        The input DataFrame to prepare
    target_column : str, optional
        Name of the target column to separate
    one_hot_encode : bool, optional
        Whether to one-hot encode categorical features

    Returns:
    --------
    df_processed: pandas.DataFrame
        The processed DataFrame
    """

    # Create a copy of the dataframe to avoid modifying the original
    df_processed = df.copy()

    # Separate target if specified
    y = None
    if target_column and target_column in df_processed.columns:
        y = df_processed[target_column].replace({"nz": 0, "hzoon": 1})
        df_processed = df_processed.drop(columns=[target_column])

    # Identify numerical and categorical columns
    numerical_cols = df_processed.select_dtypes(
        include=["int64", "float64"]
    ).columns.tolist()
    categorical_cols = df_processed.select_dtypes(
        include=["object", "category", "bool"]
    ).columns.tolist()

    # Handle missing values
    df_processed[numerical_cols] = df_processed[numerical_cols].fillna(
        df_processed[numerical_cols].median()
    )
    for col in categorical_cols:
        df_processed[col] = df_processed[col].fillna(df_processed[col].mode()[0])

    # Normalize numerical features
    if numerical_cols:
        scaler = StandardScaler()
        df_processed[numerical_cols] = scaler.fit_transform(
            df_processed[numerical_cols]
        )

    # One-hot encode categorical features
    if categorical_cols and one_hot_encode:
        df_processed = pd.get_dummies(
            df_processed, columns=categorical_cols, drop_first=False
        )

    # If we have a target column, add it back to the processed dataframe
    if target_column and y is not None:
        df_processed[target_column] = y

    return df_processed

In [75]:
def remove_unimportant_columns(dataframe, columns_to_keep):
    return dataframe[columns_to_keep]

In [76]:
data = pd.read_csv(data_file)
processed_data = remove_unimportant_columns(data, columns_to_keep)
processed_data = prepare_dataframe_for_ml(processed_data, target_column=target_column)

/var/folders/sx/g9qrr1y94z1gtsqjsbkjw2m40000gs/T/ipykernel_7872/3979001912.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = df_processed[target_column].replace({"nz": 0, "hzoon": 1})


In [77]:
processed_data.columns

Index(['DPC_FS_HA', 'CTDC_charge.G1_PB2', 'ATTCC_HA', 'TG_NS1',
       'CTDC_hydrophobicity_ARGP820101.G2_PB2', 'CAC_HA', 'DPC_MV_M1',
       'CTDT_hydrophobicity_ZIMJ680101.Tr1331_PB1',
       'CTDC_normwaalsvolume.G1_PB2', 'TA_NP',
       'CTDT_secondarystruct.Tr1221_NS1', 'PAAC_Xc1.Y_NA', 'AAGGC_M1',
       'CTDC_hydrophobicity_CASG920101.G1_PB2', 'ATTG_PA', 'TGATAC_NS1',
       'PAAC_Xc1.R_PB2', 'CTDD_secondarystruct.1.residue100_HA', 'TT_PA',
       'CTDC_polarizability.G1_PA', 'CTDT_solventaccess.Tr2332_PA',
       'CTDC_hydrophobicity_ZIMJ680101.G1_PB1', 'CTCG_PB1',
       'CTDT_normwaalsvolume.Tr1331_NA', 'AC_NA', 'GTC_NP', 'label'],
      dtype='object')

In [78]:
y = processed_data["label"]
X = processed_data.drop(columns=["label"])
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Run Models

In [79]:
def get_feature_importance(model, X):
    try:
        mdi_importances = pd.Series(
            model.feature_importances_, index=X.columns
        ).sort_values(ascending=True)
        return mdi_importances
    except AttributeError:
        pass


def get_permutation_importance(model, X, y):
    result = permutation_importance(model, X, y, n_repeats=10, random_state=42)
    # Create a Series with feature names and their mean importances
    importances = pd.Series(result.importances_mean, index=X.columns)
    # Sort importances from most to least important
    sorted_importances = importances.sort_values(ascending=False)
    return sorted_importances

In [80]:
def get_model_results(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    average_precision = average_precision_score(y_test, y_pred)
    feature_importance = get_feature_importance(model, X_test)
    permutation_importance = get_permutation_importance(model, X_test, y_test)
    permutation_importance = list(
        zip(
            permutation_importance.index,
            permutation_importance,
        )
    )
    try:
        most_important_feature = feature_importance.index[-1]
        feature_importance = list(
            zip(
                feature_importance.index,
                feature_importance,
            )
        )
    except:
        most_important_feature = None
    most_important_permutation = permutation_importance[0]
    results = {
        "accuracy": accuracy,
        "roc_auc": roc_auc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "average_precision": average_precision,
        "feature_importance": feature_importance,
        "most_important_feature": most_important_feature,
        "permutation_importance": permutation_importance,
        "most_important_permutation": most_important_permutation,
        "columns": ",".join(X_test.columns.to_list()),
    }

    return results

### XGBoost

In [81]:
xgboost = XGBClassifier()

In [82]:
results = get_model_results(xgboost, X_train, y_train, X_test, y_test)

In [83]:
results

{'accuracy': 0.9907857691323266,
 'roc_auc': np.float64(0.879286471370904),
 'precision': 0.9175257731958762,
 'recall': 0.7606837606837606,
 'f1': 0.8317757009345794,
 'average_precision': np.float64(0.7051135796871049),
 'feature_importance': [('TT_PA', 0.0002248131640953943),
  ('CTDC_hydrophobicity_ZIMJ680101.G1_PB1', 0.00023982855782378465),
  ('AC_NA', 0.0002716894377954304),
  ('CTDT_solventaccess.Tr2332_PA', 0.00027945591136813164),
  ('ATTG_PA', 0.00031795562244951725),
  ('CTDC_normwaalsvolume.G1_PB2', 0.00033543980680406094),
  ('CTDC_polarizability.G1_PA', 0.0003428725467529148),
  ('TA_NP', 0.00034452255931682885),
  ('CTDD_secondarystruct.1.residue100_HA', 0.000365082873031497),
  ('CTDT_normwaalsvolume.Tr1331_NA', 0.00040979281766340137),
  ('PAAC_Xc1.R_PB2', 0.0004680325509980321),
  ('CTDC_hydrophobicity_ARGP820101.G2_PB2', 0.0005075135850347579),
  ('AAGGC_M1', 0.0005118578555993736),
  ('CTDC_hydrophobicity_CASG920101.G1_PB2', 0.0007586886640638113),
  ('GTC_NP', 0.0

#### Plot Grid Search
parameter
- booster
- eta
- gamma
- **max_depth**
- **min_child_weight**
- max_delta_step
- **subsample**
- sampling_method
- colsample_by*
- lambda
- tree_method
- scale_pos_weight
- updater
- refresh_leaf
- process_type
- grow_policy
- max_leaves
- max_bin
- num_parallel_tree
- monotone_constraints
- interaction_constraints
- multi_strategy
- **learning_rate**

Non-exact tree methods
- max_cached_hist_node
- extmem_single_page

Dart
- sample_type
- normalize_type
- rate_drop
- one_drop
- skip_drop

In [122]:
space = {
    "n_estimators": 100 + hp.randint("n_estimators", 200),
    "max_depth": hp.quniform("max_depth", 3, 18, 1),
    # "learning_rate": hp.uniform("learning_rate", 0.01, 0.1),
    # "subsample": hp.uniform("subsample", 0.5, 1),
    # "colsample_bytree": hp.uniform("colsample_bytree", 0.1, 1),
    # "gamma": hp.uniform("gamma", 1, 9),
    # "min_child_weight": hp.quniform("min_child_weight", 0, 20, 1),
    # "reg_alpha": hp.quniform("reg_alpha", 40, 180, 1),
    # "reg_lambda": hp.uniform("reg_lambda", 0, 1),
}

In [123]:
def objective(space):
    clf = XGBClassifier(
        n_estimators=space["n_estimators"],
        max_depth=int(space["max_depth"]),
        # min_child_weight=int(space["min_child_weight"]),
        # reg_lambda=space["reg_lambda"],
        # learning_rate=space["learning_rate"],
    )

    evaluation = [(X_train, y_train), (X_test, y_test)]

    clf.fit(
        X_train,
        y_train,
    )

    pred = clf.predict(X_test)
    f1 = f1_score(y_test, pred)
    print("F1 Score:", f1)
    return {"loss": -f1, "status": "ok"}

In [124]:
best_params = fmin(objective, space, algo=tpe.suggest, max_evals=100)
print("Best set of hyperparameters: ", best_params)

F1 Score:                                              
0.827906976744186                                      
F1 Score:                                                                        
0.8207547169811321                                                               
F1 Score:                                                                        
0.8262910798122066                                                               
F1 Score:                                                                        
0.8348623853211009                                                               
F1 Score:                                                                         
0.8518518518518519                                                                
F1 Score:                                                                         
0.8411214953271028                                                                
F1 Score:                                                       

Best set of hyperparameters:  {'max_depth': np.float64(14.0), 'n_estimators': np.int64(57)}

In [ ]:
xgboost_dart = XGBClassifier(booster="dart")

In [ ]:
results_dart = get_model_results(xgboost, X_train, y_train, X_test, y_test)

In [ ]:
results_dart

### Random Forest

In [125]:
random_forest = RandomForestClassifier()

In [128]:
results = get_model_results(random_forest, X_train, y_train, X_test, y_test)

In [129]:
results

{'accuracy': 0.9905298182748912,
 'roc_auc': np.float64(0.8584466544888709),
 'precision': 0.9545454545454546,
 'recall': 0.717948717948718,
 'f1': 0.8195121951219512,
 'average_precision': np.float64(0.6937610636100526),
 'feature_importance': [('TGATAC_NS1', 0.007452999269947177),
  ('CTCG_PB1', 0.01664241439685496),
  ('GTC_NP', 0.017199234614558155),
  ('TT_PA', 0.019595684508163312),
  ('DPC_MV_M1', 0.01960318559822398),
  ('ATTG_PA', 0.020062200842379058),
  ('TA_NP', 0.020071411651786056),
  ('AAGGC_M1', 0.0205630883857929),
  ('CTDT_hydrophobicity_ZIMJ680101.Tr1331_PB1', 0.02250611672936387),
  ('CTDC_hydrophobicity_ZIMJ680101.G1_PB1', 0.023650984076453507),
  ('CTDC_hydrophobicity_ARGP820101.G2_PB2', 0.025545166041265825),
  ('CTDD_secondarystruct.1.residue100_HA', 0.025972337636008813),
  ('PAAC_Xc1.R_PB2', 0.026168310703671597),
  ('CAC_HA', 0.02746182356489744),
  ('CTDC_hydrophobicity_CASG920101.G1_PB2', 0.02822154888361058),
  ('TG_NS1', 0.03051336240011183),
  ('CTDT_nor

#### Plot Grid Search
- n_estimators
- criterion
- max_depth
- min_samples_split
- min_samples_leaf
- min_weight_fraction_leaf
- max_features
- max_leaf_nodes
- min_impurity_decrease
- bootstrap
- oob_score
- n_jobs
- random_state
- verbose
- warm_start
- class_weight
- ccp_alpha
- max_samples
- monotonic_cst

In [135]:
space = {
    "n_estimators": 100 + hp.randint("n_estimators", 200),
    "max_depth": hp.quniform("max_depth", 3, 18, 1),
    "min_samples_split": hp.quniform("min_samples_split", 2, 10, 1),
    "min_samples_leaf": hp.quniform("min_samples_leaf", 1, 10, 1),
    # "max_features": hp.choice("max_features", ["auto", "sqrt", "log2"]),
}

In [136]:
def objective(space):
    clf = RandomForestClassifier(
        n_estimators=space["n_estimators"],
        max_depth=int(space["max_depth"]),
        min_samples_split=int(space["min_samples_split"]),
        min_samples_leaf=int(space["min_samples_leaf"]),
        # max_features=space["max_features"],
    )

    evaluation = [(X_train, y_train), (X_test, y_test)]

    clf.fit(
        X_train,
        y_train,
    )

    pred = clf.predict(X_test)
    f1 = f1_score(y_test, pred)
    print("F1 Score:", f1)
    return {"loss": -f1, "status": "ok"}

In [137]:
best_params = fmin(objective, space, algo=tpe.suggest, max_evals=100)
print("Best set of hyperparameters: ", best_params)

F1 Score:                                              
0.7960199004975125                                     
F1 Score:                                                                         
0.6739130434782609                                                                
F1 Score:                                                                         
0.7319587628865979                                                                
F1 Score:                                                                         
0.7384615384615385                                                                
F1 Score:                                                                         
0.78                                                                              
F1 Score:                                                                         
0.7253886010362695                                                                
F1 Score:                                                 

Best set of hyperparameters:  {'max_depth': np.float64(18.0), 'min_samples_leaf': np.float64(1.0), 'min_samples_split': np.float64(3.0), 'n_estimators': np.int64(184)}

### Neural Network

In [ ]:
clf = TabPFNClassifier()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

### Auto

In [ ]:
clf = AutoTabPFNClassifier(max_time=120, device="cuda")  # 120 seconds tuning time
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

## Compare Model Performance
On temporally split data

In [ ]:
# load test data

## Get False Positives

## Shap Analysis